학습(train) : 정의한 모델 파라미터에 대해 학습한다.

평가(evaluate) : 학습한 모델에 대한 성능을 측정한다.

예측(Predict) : 모델을 통해 입력값에 대한 예측값을 받는다.

배포(Export):사용할 모델을 바이너리 파일로 출력한다.

features : 모델에 적용되는 입력값을 의미한다. 하나의 텐서 자료형 혹은 딕셔너리 자료형이어야 한다.

labels : 모델의 정답 라벨값을 의미한다. 하나의 텐서 자료형 혹은 딕셔너리 자료형이어야 한다. 예측과정에서는 라벨이 존재하지 않기 때문에, Estimator 에서 자동으로 이 값이 들어오지 않는다.

mode : 현재 모델 함수가 실행된 모드(학습, 검증, 예측)를 의미한다.

params : 모델에 적용될 부가적인 하이퍼파라미터 값을 의미한다.딕셔너리 자료형이어야 한다.

config : 모델에 적용할 설정값을 의미한다.

In [ ]:
# 사용법
estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = ..., configs = ... , params = ...)

In [8]:
import os
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

label = [[1],[0], [1], [1], [0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

EPOCH=100

def train_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
  dataset = dataset.repeat(EPOCH)
  dataset = dataset.batch(1)
  dataset = dataset.shuffle(len(sequences))
  iterator = dataset.make_one_shot_iterator()

  return iterator.get_next()


In [11]:
VOCAB_SIZE = len(word_index)+1
EMB_SIZE = 128

def model_fn(features, labels, mode):
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN
  EVAL = mode == tf.estimator.ModeKeys.EVAL
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT

  embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
  embed_input = tf.reduce_mean(embed_input, axis=1)

  # 은닉충을 거쳐 하나의 입력 벡터로 만들기
  hidden_layer = tf.keras.layers.Dense(128, activation='relu')(embed_input)
  output_layer = tf.keras.layers.Dense(1)(hidden_layer)
  # 출력값에 시그모이드 함수를 적용하여 0과 1사이의 값으로 만들기
  output = tf.nn.sigmoid(output_layer) 

  loss = tf.losses.mean_squared_error(output, labels)

  if TRAIN:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        train_op = train_op,
        loss = loss
        )

In [12]:
DATA_OUT_PATH = './data_out/'

import os

if not os.path.exists(DATA_OUT_PATH):
  os.makedirs(DATA_OUT_PATH)


estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')
estimator.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.